## Below code can combine two datasets by comparing the columns of 2 excel files and if the same column appears in both files, it will combine on those columns, if not then it will be empty. 
### For example, the first dataset contains 2132 rows/records, the second one contains 562 rows/records, so after combining on same columns, the final dataset should contain 2694 rows.

In [1]:
#%pip install xlsxwriter
#%pip install fuzzywuzzy
#%pip install python-Levenshtein
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher

In [2]:
def compare_and_combine_rows(file1, file2, output_file):
    # Read Excel files into pandas DataFrames
    df1 = pd.read_excel(file1)
    df2 = pd.read_excel(file2)

    # Get common columns
    common_columns = list(set(df1.columns) & set(df2.columns))

    # Convert common columns to the same data type (e.g., object)
    for col in common_columns:
        df1[col] = df1[col].astype(str)
        df2[col] = df2[col].astype(str)

    # Merge DataFrames on common columns
    merged_df = pd.merge(df1, df2, how='outer', on=common_columns)

    # Save the results to a new Excel file
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        merged_df.to_excel(writer, sheet_name='Merged', index=False)

if __name__ == "__main__":
    # Replace 'file1.xlsx', 'file2.xlsx', and 'output.xlsx' with your actual file paths
    compare_and_combine_rows('/Users/sedastepanyan/Downloads/NER_R8_step2_weightedbyfinalweight.xlsx', 
                             '/Users/sedastepanyan/Downloads/NER_step2_Flood_20231206.xlsx', 
                            '/Users/sedastepanyan/Downloads/Combined Datasets_Final.xlsx')


## Below code does the same but considers that there are columns with similar meanings but different names, so it uses fuzzy matching library fuzzywuzzy to detect these columns. 

### For example in one file the column name 'flood_sev_nodamage' is called 'flood_sev_1' in the second file. But they are essentially the same columns with different names.

In [2]:
# Function to perform sequence matching on column names
def sequence_match_ratio(col1, col2):
    return SequenceMatcher(None, col1, col2).ratio()

# Function to combine datasets based on sequence matched columns
def combine_datasets(file1, file2, output_file):
    # Read datasets from Excel files
    df1 = pd.read_excel(file1)
    df2 = pd.read_excel(file2)

    # Get column names from both datasets
    columns_df1 = df1.columns
    columns_df2 = df2.columns

    # Match columns using sequence matching
    matched_columns = []

    for col1 in columns_df1:
        best_match = max(columns_df2, key=lambda col2: sequence_match_ratio(col1, col2))
        if sequence_match_ratio(col1, best_match) > 0.8:  # Adjust the threshold as needed
            matched_columns.append((col1, best_match))

    # Convert matched columns to strings to ensure a common data type
    for col1, col2 in matched_columns:
        df1[col1] = df1[col1].astype(str)
        df2[col2] = df2[col2].astype(str)

    # Combine datasets on matched columns
    combined_df = pd.merge(df1, df2, how='outer', left_on=[col1 for col1, _ in matched_columns], right_on=[col2 for _, col2 in matched_columns])

    # Save the combined dataset to a new Excel file
    combined_df.to_excel(output_file, index=False)

    return combined_df

# Paths to the input Excel files and output Excel file
file1_path = '/Users/sedastepanyan/Downloads/NER_R8_step2_20231121.xlsx'
file2_path = '/Users/sedastepanyan/Downloads/NER_step2_Flood_20231206.xlsx'
output_path = '/Users/sedastepanyan/Downloads/Combined Datasets.xlsx'

# Combine datasets and save the result
combined_dataset = combine_datasets(file1_path, file2_path, output_path)

# Display information about the combined dataset
print(combined_dataset.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2694 entries, 0 to 2693
Columns: 765 entries, x to survey_uuid
dtypes: float64(347), object(418)
memory usage: 15.7+ MB
None
